In [ ]:
import rouskinhf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint
from scipy.stats import chi2
# import poisson confidence interval
import statsmodels.stats.proportion as smp

from scipy.stats import chi2
# import poisson confidence interval
import statsmodels.stats.proportion as smp
import plotly.express as px
from tqdm import tqdm


# Method selection

- get a distribution of the DMS signal
- for different values of p and n, compute the different confidence intervals and their performance
    - grid search for p and n
    - DMS signal distribution for p and grid search for n 


## Methods definitions
### Bootstrapping

Draw n datapoints with a probability p and take P2.5 and P97.5 as interval boundaries

In [ ]:
def bootstrapping(p, n, n_boot=10000, alpha=0.05):
    v = np.random.binomial(n, p, size=(n_boot, len(p))) / n
    return np.array(np.percentile(v, [alpha/2*100, (1-alpha/2)*100], axis=0))

### Poisson

In [ ]:

def poisson(p, n, alpha=0.05): 
    """
    uses chisquared info to get the poisson interval. Uses scipy.stats 
    (imports in function). 
    """
    if hasattr(p, '__iter__'):
        p = np.array(p)
    k = n * p
    a = alpha
    low, high = (chi2.ppf(a/2, 2*k) / 2, chi2.ppf(1-a/2, 2*k + 2) / 2)
    if hasattr(k, '__iter__'):
        low[k == 0] = 0.0
    elif k == 0: 
        low = 0.0
    return low/n, high/n

poisson([0.5], 100)

### Many methods

In [ ]:
# check the relevance of the formula
def method_factory(method):
    def method_wrapper(p, n, alpha=0.05):
        if hasattr(p, '__iter__'):
            p = np.array(p)
        return np.stack(proportion_confint(p * n, n, alpha=alpha, method=method))
    method_wrapper.__name__ = method
    return method_wrapper

methods = dict(
    poisson = poisson,
    # bootstrapping = bootstrapping,
    wilson = method_factory('wilson'),
    # binom_test = method_factory('binom_test'),
    agresti_coull = method_factory('agresti_coull'),
    jeffreys = method_factory('jeffreys'),
    normal = method_factory('normal'),
    beta = method_factory('beta')
)


# With simulated data based off real DMS distributions

In [ ]:
import pandas as pd

dms_datapoints = np.concatenate(pd.read_feather('/Users/yvesmartin/src/DMS-MaPseq_methods/df.feather').drop_duplicates('sequence')['sub_rate'].values)
dms_datapoints = dms_datapoints[dms_datapoints != -1000.]

plt.hist(dms_datapoints, bins=100)
plt.show()

In [ ]:

# no seed for np
np.random.seed(np.random.randint(0, 1000000))

alpha = 0.95

def subsampling(p, n):
    return np.random.binomial(n, p) / n

n = 1000
N = 1000
n_experiments = 10
is_in_interval = []

for _ in tqdm(range(n_experiments), total = n_experiments):
    for n in [100, 200, 500, 1000, 1500, 2000, 5000, 10000]:
        true = np.random.choice(dms_datapoints, N) 
        subsample = subsampling(true, n)
        for name, method in methods.items():
            interval = method(subsample, n)
            for p, min, max in zip(subsample, interval[0], interval[1]):
                if p < 0.20:
                    is_in_interval.append({
                        'n': str(n),
                        'p': p,
                        'p_interval': '[{}, {}]'.format(np.floor(p*100)/100, (1+np.floor(p*100))/100),
                        'method': name,
                        'score': sum((interval[0] <= true) & (true <= interval[1])) / len(true),
                        'width': np.mean(interval[1] - interval[0]),
                        'min': np.mean(interval[0]),
                        'max': np.mean(interval[1])
                    })

is_in_interval = pd.DataFrame(is_in_interval)


In [ ]:
# df = pd.read_feather('simu.feather')
df = is_in_interval
# px.box(is_in_interval, x='round_p', y='score', color='method').show()
for n, g in df.groupby('n'):
    df.loc[df['n'] == n, 'n'] = '{:,} reads (n={:,})'.format(int(n), len(g))

for p, g in df.groupby('p_interval'):
    df.loc[df['p_interval'] == p, 'p_interval'] = '{} (n={:,})'.format(p, len(g))

In [ ]:
fig = px.box(df, x='n', y='score', color='method')
fig.update_layout(
    xaxis_title="number of reads in the sub-sample",
    yaxis_title="% of the true values in the confidence interval",  
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    ),
    height=800, 
    width = 1200,
    #background color
    plot_bgcolor='rgba(0,0,0,0)',
)
# add frame
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2
    ),
    yaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2
    )
)

# add red line at 0.95
fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=-0.5,
            y0=0.95,
            x1=len(df['n'].unique())-0.5,
            y1=0.95,
            line=dict(
                color="Red",
                width=3,
                dash="dashdot",
            )
))

fig.show()

# save to pdf
import plotly.io as pio
pio.write_image(fig, 'images/ci_n.pdf')

In [ ]:
fig = px.box(df.sort_values('p_interval'), x='p_interval', y='score', color='method', points=None,)
fig.update_layout(
    xaxis_title="mutation fraction intervals",
    yaxis_title="% of the true values in the conf. interval",  
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    ),
    height=800, 
    width = 1200,
    #background color
    plot_bgcolor='rgba(0,0,0,0)',
)
# add frame
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        tickangle=90,
    ),
    yaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2
    )
)


# add red line at 0.95
fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=-0.5,
            y0=0.95,
            x1=len(df['p_interval'].unique())-0.5,
            y1=0.95,
            line=dict(
                color="Red",
                width=3,
                dash="dashdot",
            )
))


fig.show()

# to pdf
import plotly.io as pio
pio.write_image(fig, 'images/ci_p.pdf')